In [ ]:
import os
from datetime import datetime
from typing import Literal

import torch
import wandb
import numpy as np
from torch import nn
from torch import optim
import torch_geometric.nn as geonn
from tqdm import tqdm

from gfos.loss import MultiElementRankLoss
from gfos.utils.misc import seed_everything


SEED = 42
DEBUG = False

seed_everything(SEED)
device = "cuda" if torch.cuda.is_available() else "cpu"

os.environ["WANDB_NOTEBOOK_NAME"] = "tile_gnn.ipynb"


## Configs

In [ ]:
TILE_DIR = "../../data/npz_all/npz/tile/xla/"

configs = dict(
    conv_layer="GATConv",
    num_epochs=1000,
    graph_dim=128,
    config_dim=128,
    learning_rate=1e-3,
    weight_decay=1e-7,
    min_lr=1e-7,
    max_configs=8192,
    num_configs=1024,
    loss_margin=0.5,
    loss_num_permutations=200,
)

WANDB_PROJECT = "gfos"
WANDB_DIR = "../../logs/"
WANDB_RUN_NAME = "tile_xla_mse"
TAGS = ["train", "tile", "xla"]

NUM_VAL_EPOCHS = 8
INFERENCE_CONFIGS_BATCH_SIZE = 500


## Model

In [ ]:
class LayoutModel(torch.nn.Module):
    def __init__(
        self,
        conv_layer: Literal["GATConv", "GCNConv", "SAGEConv"],
        graph_dim: int = 64,
        config_dim: int = 64,
        num_node_feats: int = 140,
        num_config_feats: int = 24,
    ):
        super().__init__()

        conv_layer = getattr(geonn, conv_layer)

        op_embedding_dim = 32
        merged_node_dim = graph_dim + config_dim

        self.embedding = torch.nn.Embedding(
            120,
            op_embedding_dim,
        )
        in_channels = op_embedding_dim + num_node_feats

        self.convs = geonn.Sequential('x, edge_index', [
            # (nn.Dropout(p=0.1), 'x -> x'),
            (conv_layer(in_channels, graph_dim), 'x, edge_index -> x1'),
            nn.LeakyReLU(inplace=True),
            (conv_layer(graph_dim, graph_dim), 'x1, edge_index -> x2'),
            # (lambda x1, x2: x1 + x2, 'x1, x2 -> x3'),
            nn.LeakyReLU(inplace=True),
            (conv_layer(graph_dim, graph_dim), 'x2, edge_index -> x4'),
            nn.LeakyReLU(inplace=True),
            (conv_layer(graph_dim, graph_dim), 'x4, edge_index -> x5'),
            # (lambda x4, x5: x4 + x5, 'x4, x5 -> x6'),
            nn.LeakyReLU(inplace=True),
            # (conv_layer(hidden_channels[3], hidden_channels[3]), 'x5, edge_index -> x6'),
            # nn.LeakyReLU(inplace=True),
            # (conv_layer(hidden_channels[3], hidden_channels[4]), 'x6, edge_index -> x7'),
            # nn.LeakyReLU(inplace=True),
            # (lambda x4, x5: x4 + x5, 'x5, x7 -> x8'),
        ])

        self.config_prj = nn.Sequential(
            nn.Linear(num_config_feats, config_dim),
            nn.LeakyReLU(),
        )

        # Define a sequential dense neural network
        self.dense = torch.nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(merged_node_dim, 64, bias=False),
            nn.LeakyReLU(),
            nn.Linear(64, 64, bias=False),
            nn.LeakyReLU(),
            nn.Linear(64, 1, bias=False),
        )

    def forward(
        self,
        node_feat: torch.Tensor,
        node_opcode: torch.Tensor,
        edge_index: torch.Tensor,
        config_feat: torch.Tensor,
    ) -> torch.Tensor:
        c = config_feat.size(0)

        x = torch.cat([node_feat, self.embedding(node_opcode)], dim=1)

        x = self.convs(x, edge_index)
        x = x.mean(dim=0).repeat((c, 1))
        config_feat = self.config_prj(config_feat)

        x = torch.cat([x, config_feat], dim=-1)
        x = nn.functional.normalize(x, dim=-1)

        x = self.dense(x).flatten()
        
        # x = torch.sigmoid(x)

        return x

In [ ]:
from pathlib import Path
from torch.utils.data import Dataset
from dataclasses import dataclass

@dataclass
class Normalizer:
    node_feat_mask: torch.Tensor
    node_feat_min: torch.Tensor
    node_feat_max: torch.Tensor
    node_config_feat_mask: torch.Tensor
    node_config_feat_min: torch.Tensor
    node_config_feat_max: torch.Tensor

    def normalize_node_feat(self, node_feat: torch.Tensor) -> torch.Tensor:
        assert node_feat.ndim == 2, "node_feat must be 2D"
        node_feat = node_feat[:, self.node_feat_mask]

        return (node_feat - self.node_feat_min) / (
            self.node_feat_max - self.node_feat_min
        )

    def normalize_config_feat(
        self, config_feat: torch.Tensor
    ) -> torch.Tensor:
        assert config_feat.ndim == 2, "node_config_feat must be 2D"
        config_feat = config_feat[:, self.node_config_feat_mask]
        return (config_feat - self.node_config_feat_min) / (
            self.node_config_feat_max - self.node_config_feat_min
        )

    @classmethod
    def from_dict(
        cls,
        configs: dict,
    ) -> "Normalizer":
    
        node_feat_mask = torch.tensor(
            configs["node_feat_mask"], dtype=torch.bool
        )
        node_feat_min = torch.tensor(
            configs["node_feat_min"], dtype=torch.float
        )[node_feat_mask]
        node_feat_max = torch.tensor(
            configs["node_feat_max"], dtype=torch.float
        )[node_feat_mask]
        config_feat_mask = torch.tensor(
            configs["config_feat_mask"], dtype=torch.bool
        )
        config_feat_min = torch.tensor(
            configs["config_feat_min"], dtype=torch.float
        )[config_feat_mask]
        config_feat_max = torch.tensor(
            configs["config_feat_max"], dtype=torch.float
        )[config_feat_mask]

        return Normalizer(
            node_feat_mask=node_feat_mask,
            node_feat_min=node_feat_min,
            node_feat_max=node_feat_max,
            node_config_feat_mask=config_feat_mask,
            node_config_feat_min=config_feat_min,
            node_config_feat_max=config_feat_max,
        )

    @classmethod
    def from_json(cls, path):
        import json

        json_data = json.load(open(path))
        return Normalizer.from_dict(json_data)


class TileDataset(Dataset):
    """Load all data in advance."""

    def __init__(
        self,
        files: list[str],
        max_configs: int = -1,
        num_configs: int = -1,
        normalizer: Normalizer = None,
        test: bool = False,  # TODO: remove
    ):
        self.max_configs = max_configs
        self.num_configs = num_configs
        self.files = files
        self.normalizer = normalizer

        self.data = []
        
        self.files = [f for f in self.files if np.load(f)['config_runtime'].shape[0] > 10]

        for file in tqdm(self.files, desc="Loading data"):
            record = dict(np.load(file))
            model_id = Path(file).stem
            record["model_id"] = model_id
            runtime = record["config_runtime"]
            record["original_config_runtime"] = runtime.copy()

            config_runtime_normalizers = record["config_runtime_normalizers"]
            runtime = runtime / (config_runtime_normalizers + 1e-10)
            runtime = (runtime - runtime.mean()) / runtime.std()

            # TODO: invest when sampling results in better kendall scores on validation set
            # and scores after sampling are close LB scores
            # Currently, use sampling for both training and validation
            if not test:
                runtime_sampled, config_indices = sample_configs(
                    runtime, max_configs
                )
            else:
                runtime_sampled = runtime
                config_indices = torch.arange(len(runtime))

            record["config_runtime"] = runtime_sampled
            record["config_feat"] = record["config_feat"][
                config_indices
            ]
            record["original_config_runtime"] = record[
                "original_config_runtime"
            ][config_indices]

            self.data.append(record)

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        record = self.data[idx]
        config_runtime = torch.tensor(
            record["config_runtime"], dtype=torch.float
        )

        if self.num_configs > 0:
            num_configs = self.num_configs
        elif self.max_configs > 0:
            num_configs = self.max_configs
        else:
            num_configs = config_runtime.size(0)

        # Shuffle
        if self.max_configs > 0 or self.num_configs > 0:
            config_indices = torch.randperm(config_runtime.size(0))[
                :num_configs
            ]
        else:
            config_indices = torch.arange(num_configs)
        config_runtime = config_runtime[config_indices]
        original_config_runtime = torch.tensor(
            record["original_config_runtime"], dtype=torch.float
        )[config_indices]

        model_id = record["model_id"]
        node_feat = torch.tensor(record["node_feat"], dtype=torch.float)
        node_opcode = torch.tensor(record["node_opcode"], dtype=torch.long)
        edge_index = torch.tensor(
            np.swapaxes(record["edge_index"], 0, 1), dtype=torch.long
        )

        config_feat = torch.tensor(
            record["config_feat"], dtype=torch.float
        )
        config_feat = config_feat[config_indices]

        if self.normalizer is not None:
            node_feat = self.normalizer.normalize_node_feat(node_feat)
            config_feat = self.normalizer.normalize_config_feat(
                config_feat
            )

        sample = dict(
            model_id=model_id,
            node_feat=node_feat,
            node_opcode=node_opcode,
            edge_index=edge_index,
            config_feat=config_feat,
            config_runtime=config_runtime,
            original_config_runtime=original_config_runtime,
        )

        return sample
    
    
def sample_configs(
    config_runtime: np.array, max_configs: int
) -> (np.array, np.array):
    """Sample 1/3 max_configs of best configs and 1/3 of worst configs,
    and the rest randomly. Return the sampled configs and indices.
    """
    c = len(config_runtime)
    max_configs = min(max_configs, c) if max_configs > 0 else c
    third = max_configs // 3

    sorted_indices = np.argsort(config_runtime)

    keep_indices = np.concatenate(
        [
            sorted_indices[:third],  # Good configs.
            sorted_indices[-third:],  # Bad configs.
            np.random.choice(
                sorted_indices[third:-third],
                max_configs - 2 * third,
            ),
        ]
    )

    return config_runtime[keep_indices], keep_indices


## Prepare for training

### Expand configs

In [ ]:
conv_layer = configs["conv_layer"]
num_epochs = configs["num_epochs"]
graph_dim = configs["graph_dim"]
config_dim = configs["config_dim"]
learning_rate = configs["learning_rate"]
weight_decay = configs["weight_decay"]
min_lr = configs["min_lr"]
max_configs = configs["max_configs"]
num_configs = configs["num_configs"]
margin = configs["loss_margin"]
number_permutations = configs["loss_num_permutations"]


### Model, loss, optimizer, scheduler

In [ ]:
# Read data
tile_dir = Path(TILE_DIR)

train_files = list(tile_dir.glob("train/*.npz"))
valid_files = list(tile_dir.glob("valid/*.npz"))

normalizer = Normalizer.from_json(
    "../../data/tile_normalizers.json",
)

train_dataset = TileDataset(train_files, max_configs=max_configs, num_configs=num_configs, normalizer=normalizer)
valid_dataset = TileDataset(valid_files, normalizer=normalizer)


In [ ]:
num_node_feats = train_dataset[0]["node_feat"].size(1)
num_config_feats = train_dataset[0]["config_feat"].size(1)

In [ ]:
model = LayoutModel(
    conv_layer=conv_layer,
    graph_dim=64,
    config_dim=64,
    num_config_feats=num_config_feats,
    num_node_feats=num_node_feats,
).to(device)

criterion = MultiElementRankLoss(
    margin=margin, number_permutations=number_permutations
)

# criterion = nn.MSELoss()

# optimizer = optim.Adam([
#     {"name": "lr_embed", 'params': model.embedding.parameters(), 'lr': learning_rate_slow},
#     {"name": "lr_config", 'params': model.config_prj.parameters(), 'lr': learning_rate_slow},
#     {"name": "lr_gnn", 'params': model.convs.parameters(), 'lr': learning_rate_slow},
#     {"name": "lr_deg", 'params': model.deg_prj.parameters(), 'lr': learning_rate_slow},
#     {"name": "lr_encoder", 'params': model.encoder.parameters(), 'lr': learning_rate_fast},
#     {"name": "lr_dense", 'params': model.dense.parameters(), 'lr': learning_rate_fast},
# ])

optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode="min", factor=0.1, patience=3, min_lr=min_lr
)

# Split data

### Init wandb and train&valid dataset

In [ ]:
if not DEBUG:
    run = wandb.init(
        project=WANDB_PROJECT,
        dir=WANDB_DIR,
        name=WANDB_RUN_NAME,
        config=configs,
        tags=TAGS,
    )
    run.watch(model, log="all")
    run.log_code("../")

    time_str = datetime.now().strftime("%Y%m%d_%H%M%S")
    log_dir = f"../../logs/{WANDB_RUN_NAME}/{time_str}"
    os.makedirs(log_dir, exist_ok=True)

best_score = -1


## Training

In [ ]:
def tile_topk(preds: torch.Tensor, targets: torch.Tensor):
    predbest_idx = torch.topk(preds, k=5, largest=False)[1]
    predbest = targets[predbest_idx].min()
    allbest = targets.min()
    
    return 2 - predbest / allbest

In [ ]:
accum_iter = 16
loss_mean = 0

for epoch in range(num_epochs):
    # Shuffle the training dataset
    permutation = np.random.permutation(len(train_dataset))

    # Training phase
    model.train()
    pbar = tqdm(permutation, leave=False)

    for i in pbar:
        record = train_dataset[i]
        node_feat = record["node_feat"]
        node_opcode = record["node_opcode"]
        edge_index = record["edge_index"]
        config_feat = record["config_feat"]
        config_runtime = record["config_runtime"]

        (
            node_feat,
            node_opcode,
            edge_index,
            config_feat,
            config_runtime,
        ) = (
            node_feat.to(device),
            node_opcode.to(device),
            edge_index.to(device),
            config_feat.to(device),
            config_runtime.to(device),
        )

        out = model(
            node_feat,
            node_opcode,
            edge_index,
            config_feat,
        )

        loss = criterion(out, config_runtime)
        
        loss = loss / accum_iter
        loss_mean += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1e-2)

        if ((i + 1) % accum_iter == 0) or (i + 1 == len(train_dataset)):
            optimizer.step()
            optimizer.zero_grad()

            if not DEBUG:
                wandb.log(
                    {
                        "epoch": epoch,
                        "train/lr_slow": optimizer.param_groups[0]["lr"],
                        "train/loss": loss_mean,
                    }
                )
            loss_mean = 0

        pbar.set_description(f"epoch: {epoch} loss: {(loss.item()):.4f}")

    pbar.close()

    if (epoch + 1) % NUM_VAL_EPOCHS != 0 and epoch != num_epochs - 1:
        continue

    model.eval()

    # Validation phase
    # Scores placeholder
    topk_scores = []

    with torch.no_grad():
        for record in tqdm(valid_dataset, desc="valid", leave=False):
            node_feat = record["node_feat"]
            node_opcode = record["node_opcode"]
            edge_index = record["edge_index"]
            config_feat = record["config_feat"]
            original_config_runtime = record["original_config_runtime"]

            (
                node_feat,
                node_opcode,
                edge_index,
                config_feat,
            ) = (
                node_feat.to(device),
                node_opcode.to(device),
                edge_index.to(device),
                config_feat.to(device),
            )
            num_configs = original_config_runtime.shape[-1]
            outs = []

            for i in range(0, num_configs, INFERENCE_CONFIGS_BATCH_SIZE):
                end_i = min(i + INFERENCE_CONFIGS_BATCH_SIZE, num_configs)
                out: torch.Tensor = model(
                    node_feat,
                    node_opcode,
                    edge_index,
                    config_feat[i:end_i],
                )
                outs.append(out.detach().cpu())

            outs = torch.concat(outs)
            topk_scores.append(tile_topk(outs, original_config_runtime).item())

    topk_mean = np.mean(topk_scores)
    scheduler.step(topk_mean)
    

    if not DEBUG:
        wandb.log(
            {
                "val/top5": topk_mean,
            }
        )

    print(
        f"epoch {epoch}, top5 = {topk_mean:.4f}"
    )

    # Update best scores and save the model if the mean score improves
    if topk_mean > best_score:
        best_score = topk_mean
        print(f"Best score updated: {best_score:.4f}")
        if not DEBUG:
            filename = f"{epoch}_{best_score:.4f}.pth"
            path = os.path.join(wandb.run.dir, filename)
            torch.save(
                model.state_dict(),
                path,
            )

if not DEBUG:
    run.finish()

In [ ]:
wandb.finish()